In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import urllib2
import random
from json import loads
import re

headerstr = '''User-Agent:Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50
User-Agent:Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50
User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0
User-Agent:Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)
User-Agent:Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)
User-Agent: Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1
User-Agent:Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1
'''

def headers():
    header = headerstr.split('\n')
    length = len(header)
    return header[random.randint(0,length -1)]

def getUrlList():
    req = urllib2.Request(
        'https://mm.taobao.com/tstar/search/tstar_model.do?_input_charset=utf-8')
    req.add_header('User-Agent',headers())
    html = urllib2.urlopen(
        req,data='q=&viewFlag=A&sortType=default&searchStyle=&searchRegion=city%3A&searchFansNum=&currentPage=1&pageSize=100').read(
    ).decode('gbk').encode('utf-8')
    #print(html)
    json = loads(html)
    return json['data']['searchDOList']
       
def getInfo(userId):
    req = urllib2.Request(
        'https://mm.taobao.com/self/aiShow.htm?spm=719.7763510.1998643336.1.DNemgU&userId=%s' %userId)
    req.add_header('User-Agent',headers())
    html = urllib2.urlopen(req).read().decode('gbk').encode('utf-8')
    #print(html)

def getAlbumList(userId):
    req = urllib2.Request(
        'https://mm.taobao.com/self/album/open_album_list.htm?_charset=utf-8&user_id=%s' %userId)
    req.add_header('User-Agent',headers())
    html = urllib2.urlopen(req).read().decode('gbk').encode('utf-8')
    #print(html)
    reg = r'class="mm-first" href="//(.*?)"'
    return re.findall(reg,html)[::2]

def getImageUrl(url):
    req = urllib2.Request(url)
    req.add_header('User-Agent',headers())
    html = urllib2.urlopen(req).read().decode('gbk').encode('utf-8')
    #print html
    #reg = r'class="mm-template-src" src="//(.*?)"'
    #return re.findall(reg,html)
    
for i in getUrlList():
    userId = (i['userId'])
    #getInfo(userId)
    for n in getAlbumList(userId):
        url = "https://"+n
        print(url)
        #print(getImageUrl(url))
        #getImageUrl(url)
    break     

https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10000962815&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10000794223&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10000878652&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=77593777&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10651284&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=300615106&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10000020885&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=10000002129&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=301786011&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&album_id=301785175&album_flag=0
https://mm.taobao.com/self/album_photo.htm?user_id=176817195&a

# 抓取淘宝MM照片

*    目标   
     > 1.抓取淘宝MM的姓名，头像，年龄

     > 2.抓取每一个MM的资料简介以及写真图片

     > 3.把每一个MM的写真图片按照文件夹保存到本地

     > 4.熟悉文件保存的过程

## URL的格式
在这里我们用到的URL是 http://mm.taobao.com/json/request_top_list.htm?page=1 ，问号前面是基地址，后面的参数page是代表第几页，可以随意更换地址。点击开之后，会发现有一些淘宝MM的简介，并附有超链接链接到个人详情页面。

我们需要抓取本页面的头像地址，MM姓名，MM年龄，MM居住地，以及MM的个人详情页面地址。

## 抓取简要信息
我们首先抓取本页面的MM详情页面地址，姓名，年龄等等的信息打印出来，直接贴代码如下:

In [1]:
# -*- coding:utf-8 -*-
 
import urllib
import urllib2
import re
 
class Spider:
 
    def __init__(self):
        self.siteURL = 'http://mm.taobao.com/json/request_top_list.htm'
 
    def getPage(self,pageIndex):
        url = self.siteURL + "?page=" + str(pageIndex)
        print url
        request = urllib2.Request(url)
        response = urllib2.urlopen(request)
        return response.read().decode('gbk').encode('utf-8')
 
    def getContents(self,pageIndex):
        page = self.getPage(pageIndex)
        pattern = re.compile('<div class="list-item".*?pic-word.*?<a href="(.*?)".*?<img src="(.*?)".*?<a class="lady-name.*?>(.*?)</a>.*?<strong>(.*?)</strong>.*?<span>(.*?)</span>',re.S)
        items = re.findall(pattern,page)
        for item in items:
            print item[0],item[1],item[2],item[3],item[4]
 
spider = Spider()
spider.getContents(1)

http://mm.taobao.com/json/request_top_list.htm?page=1
//mm.taobao.com/687471686.htm //gtd.alicdn.com/sns_logo/i2/TB1XZ1PQVXXXXaJXpXXSutbFXXX.jpg_60x60.jpg 田媛媛 27 广州市
//mm.taobao.com/405095521.htm //gtd.alicdn.com/sns_logo/i5/TB1nK_zJXXXXXaaXVXXSutbFXXX.jpg_60x60.jpg v悦悦 27 杭州市
//mm.taobao.com/631300490.htm //gtd.alicdn.com/sns_logo/i8/TB1JRHEQpXXXXXwXVXXSutbFXXX.jpg_60x60.jpg 崔辰辰 28 杭州市
//mm.taobao.com/414457129.htm //gtd.alicdn.com/sns_logo/i6/TB1uGkBKVXXXXXiXFXXSutbFXXX.jpg_60x60.jpg 大猫儿 31 广州市
//mm.taobao.com/141234233.htm //gtd.alicdn.com/sns_logo/i8/TB1TVQ_LXXXXXcpXXXXSutbFXXX.jpg_60x60.jpg 金甜甜 1 杭州市
//mm.taobao.com/96614110.htm //gtd.alicdn.com/sns_logo/i1/TB1zh6tRVXXXXclaXXXSutbFXXX.jpg_60x60.jpg 紫轩 30 杭州市
//mm.taobao.com/37448401.htm //gtd.alicdn.com/sns_logo/i8/TB1bU.UFFXXXXc0XpXXSutbFXXX.jpg_60x60.jpg 谢婷婷 28 杭州市
//mm.taobao.com/74386764.htm //gtd.alicdn.com/sns_logo/i8/TB1HmB7GXXXXXbHXFXXSutbFXXX.jpg_60x60.jpg 夏晨洁 28 杭州市
//mm.taobao.com/523216808.htm //gtd.alicdn.com/sns_logo

## 文件写入简介
在这里，我们有写入图片和写入文本两种方式
### 写入图片

In [ ]:
import urllib

u = urllib.urlopen('https://gtd.alicdn.com/sns_logo/i2/T15LjwXkpgXXaCwpjX.png')
data = u.read()
f = open('/home/zhangwei/Pictures/icon.png', 'wb')
f.write(data)
print u"正在悄悄保存她的一张图片为",'/home/zhangwei/Pictures/icon.png'
f.close(

### 写入文本

In [ ]:
def saveBrief(self,content,name):
    fileName = name + "/" + name + ".txt"
    f = open(fileName,"w+")
    print (u"正在偷偷保存她的个人信息为",fileName)
    f.write(content.encode('utf-8'))

### 创建新目录

In [ ]:
#创建新目录
def mkdir(self,path):
    path = path.strip()
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists=os.path.exists(path)
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(path)
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        return False

## 代码完善

In [21]:
# -*- coding:utf-8 -*-
 
import urllib
import urllib2
import re
import os
import random

#反爬
headerstr = '''User-Agent:Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50
User-Agent:Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50
User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0
User-Agent:Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)
User-Agent:Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)
User-Agent: Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1
User-Agent:Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1
'''

def headers():
    header = headerstr.split('\n')
    length = len(header)
    return header[random.randint(0,length -1)]
 
#处理页面标签类
class Tool:
    #去除img标签,1-7位空格, 
    removeImg = re.compile('<img.*?>| {1,7}| ')
    #删除超链接标签
    removeAddr = re.compile('<a.*?>|</a>')
    #把换行的标签换为\n
    replaceLine = re.compile('<tr>|<div>|</div>|</p>')
    #将表格制表<td>替换为\t
    replaceTD= re.compile('<td>')
    #将换行符或双换行符替换为\n
    replaceBR = re.compile('<br><br>|<br>')
    #将其余标签剔除
    removeExtraTag = re.compile('<.*?>')
    #将多行空行删除
    removeNoneLine = re.compile('\n+')
    def replace(self,x):
        x = re.sub(self.removeImg,"",x)
        x = re.sub(self.removeAddr,"",x)
        x = re.sub(self.replaceLine,"\n",x)
        x = re.sub(self.replaceTD,"\t",x)
        x = re.sub(self.replaceBR,"\n",x)
        x = re.sub(self.removeExtraTag,"",x)
        x = re.sub(self.removeNoneLine,"\n",x)
        #strip()将前后多余内容删除
        return x.strip()
 
 #抓取MM
class Spider:
 
    #页面初始化
    def __init__(self):
        self.siteURL = 'http://mm.taobao.com/json/request_top_list.htm'
        self.tool = Tool()
 
    #获取索引页面的内容
    def getPage(self,pageIndex):
        url = self.siteURL + "?page=" + str(pageIndex)
        request = urllib2.Request(url)
        response = urllib2.urlopen(request)
        return response.read().decode('gbk').encode('utf-8')
 
    #获取索引界面所有MM的信息，list格式
    def getContents(self,pageIndex):
        page = self.getPage(pageIndex)
        pattern = re.compile('<div class="list-item".*?pic-word.*?<a href="(.*?)".*?<img src="(.*?)".*?<a class="lady-name.*?>(.*?)</a>.*?<strong>(.*?)</strong>.*?<span>(.*?)</span>',re.S)
        items = re.findall(pattern,page)
        contents = []
        for item in items:
            contents.append([item[0],item[1],item[2],item[3],item[4]])
        return contents
 
    #获取MM个人详情页面
    def getDetailPage(self,infoURL):
        request = urllib2.Request(infoURL)
        request.add_header('User-Agent',headers())
        response = urllib2.urlopen(request)
        return response.read().decode('gbk').encode('utf-8')
 
    #获取个人文字简介
    def getBrief(self,page):
        pattern = re.compile('<div class="mm-aixiu-content".*?>(.*?)<!--',re.S)
        result = re.search(pattern,page)
        if result is None:
            pass
        else:
            return self.tool.replace(result.group(1))
 
    #获取页面所有图片
    def getAllImg(self,page):
        pattern = re.compile('<div class="mm-aixiu-content".*?>(.*?)<!--',re.S)
        #个人信息页面所有代码
        content = re.search(pattern,page)
        #从代码中提取图片
        patternImg = re.compile('<img.*?src="(.*?)"',re.S)
        if content is None:
            pass
        else:
            return re.findall(patternImg,content.group(1))
 
    #保存多张写真图片
    def saveImgs(self,images,name):
        number = 1
        print u"发现",name,u"共有",len(images),u"张照片"
        for imageURL in images:
            splitPath = imageURL.split('.')
            fTail = splitPath.pop()
            if len(fTail) > 3:
                fTail = "jpg"
            fileName = name + "/" + str(number) + "." + fTail
            self.saveImg(imageURL,fileName)
            number += 1
 
    # 保存头像
    def saveIcon(self,iconURL,name):
        iconURL = iconURL[:-10]
        splitPath = iconURL.split('.')
        fTail = splitPath.pop()
        fileName = '/home/zhangwei/Jupyter/Web Spider/'+ name + "/icon." + fTail
        #print iconURL,fileName
        self.saveImg(iconURL,fileName)
 
    #保存个人简介
    def saveBrief(self,content,name):
        fileName = name + "/" + name + ".txt"
        f = open(fileName,"w+")
        print u"正在偷偷保存她的个人信息为",fileName
        f.write(content.encode('utf-8'))
 
    #传入图片地址，文件名，保存单张图片
    def saveImg(self,imageURL,fileName):
         u = urllib.urlopen(imageURL)
         data = u.read()
         f = open(fileName, 'wb')
         f.write(data)
         print u"正在悄悄保存她的一张图片为",fileName
         f.close()
 
    #创建新目录
    def mkdir(self,path):
        path = path.strip()
        path = '/home/zhangwei/Jupyter/Web Spider/' + path
        # 判断路径是否存在
        # 存在     True
        # 不存在   False
        isExists=os.path.exists(path)
        # 判断结果
        if not isExists:
            # 如果不存在则创建目录
            print u"偷偷新建了名字叫做",path,u'的文件夹'
            # 创建目录操作函数
            os.makedirs(path)
            return True
        else:
            # 如果目录存在则不创建，并提示目录已存在
            print u"名为",path,'的文件夹已经创建成功'
            return False
 
    #将一页淘宝MM的信息保存起来
    def savePageInfo(self,pageIndex):
        #获取第一页淘宝MM列表
        contents = self.getContents(pageIndex)
        for item in contents:
            item[0] = "https:"+ item[0]
            item[1] = "https:"+ item[1]
            #item[0]个人详情URL,item[1]头像URL,item[2]姓名,item[3]年龄,item[4]居住地
            print u"发现一位模特,名字叫",item[2],u"芳龄",item[3],u",她在",item[4]
            print u"正在偷偷地保存",item[2],"的信息"
            print u"又意外地发现她的个人地址是",item[0]
            print u"个人头像URL",item[1]
            #个人详情页面的URL
            detailURL = item[0]
            #得到个人详情页面代码
            detailPage = self.getDetailPage(detailURL)
            #获取个人简介
            brief = self.getBrief(detailPage)
            #获取所有图片列表
            images = self.getAllImg(detailPage)
            self.mkdir(item[2])
#             #保存个人简介
#             self.saveBrief(brief,item[2])
            #保存头像
            self.saveIcon(item[1],item[2])
#             #保存图片
#             self.saveImgs(images,item[2])
 
    #传入起止页码，获取MM图片
    def savePagesInfo(self,start,end):
        for i in range(start,end+1):
            print u"正在偷偷寻找第",i,u"个地方，看看MM们在不在"
            self.savePageInfo(i)
 
 #传入起止页码即可，在此传入了2,10,表示抓取第2到10页的MM
spider = Spider()
spider.savePagesInfo(2,3)

正在偷偷寻找第 2 个地方，看看MM们在不在
发现一位模特,名字叫 李喵喵 芳龄 27 ,她在 上海市
正在偷偷地保存 李喵喵 的信息
又意外地发现她的个人地址是 https://mm.taobao.com/45834230.htm
个人头像URL https://gtd.alicdn.com/sns_logo/i2/T15LjwXkpgXXaCwpjX.png_60x60.jpg
名为 /home/zhangwei/Jupyter/Web Spider/李喵喵 的文件夹已经创建成功
正在悄悄保存她的一张图片为 /home/zhangwei/Jupyter/Web Spider/李喵喵/icon.png
发现一位模特,名字叫 悦小舞 芳龄 27 ,她在 北京市
正在偷偷地保存 悦小舞 的信息
又意外地发现她的个人地址是 https://mm.taobao.com/91442126.htm
个人头像URL https://gtd.alicdn.com/sns_logo/i8/T1kYSMXxBaXXb1upjX.jpg_60x60.jpg
名为 /home/zhangwei/Jupyter/Web Spider/悦小舞 的文件夹已经创建成功
正在悄悄保存她的一张图片为 /home/zhangwei/Jupyter/Web Spider/悦小舞/icon.jpg
发现一位模特,名字叫 夏欢欢 芳龄 27 ,她在 杭州市
正在偷偷地保存 夏欢欢 的信息
又意外地发现她的个人地址是 https://mm.taobao.com/272536951.htm
个人头像URL https://gtd.alicdn.com/sns_logo/i8/TB1MUkbLFXXXXXYaXXXSutbFXXX.jpg_60x60.jpg
名为 /home/zhangwei/Jupyter/Web Spider/夏欢欢 的文件夹已经创建成功
正在悄悄保存她的一张图片为 /home/zhangwei/Jupyter/Web Spider/夏欢欢/icon.jpg
发现一位模特,名字叫 赵颖 芳龄 27 ,她在 广州市
正在偷偷地保存 赵颖 的信息
又意外地发现她的个人地址是 https://mm.taobao.com/63153008.htm
个人头像URL https://gtd.alicdn

In [18]:
import urllib

u = urllib.urlopen('https://gtd.alicdn.com/sns_logo/i2/T15LjwXkpgXXaCwpjX.png')
data = u.read()
f = open('/home/zhangwei/Pictures/icon.png', 'wb')
f.write(data)
print u"正在悄悄保存她的一张图片为",'/home/zhangwei/Pictures/icon.png'
f.close()

正在悄悄保存她的一张图片为 /home/zhangwei/Pictures/icon.png


#  python学习（7）：python爬虫之爬取动态加载的图片，以百度图片为例 

## 前言
前面我们爬取图片的网站都是静态的，在页面中右键查看源码就能看到网页中图片的位置。这样我们用requests库得到页面源码后，再用bs4库解析标签即可保存图片到本地。

当我们在看百度图片时，右键–检查–Elements，点击箭头，再用箭头点击图片时，会显示图片的位置和样式。但是，当我们右键查看网页源码时，出来的却是一大堆JavaScript代码，并没有图片的链接等信息。这是为什么呢？
![](RE04.png)
![](RE05.png)
这是因为，百度图片的网页是一个动态页面，它的网页原始数据其实是没有这个图片的，通过运行javascript，把这个图片数据把它插入到网页的html标签里面，那这样造成的结果是，我们在开发者工具中虽然能看到这个html标签，但实际上，当我们在看网页的原始数据的时候，其实是没有这个标签的，它只在运行时加载和渲染，那这个时候怎么办呢？怎么把这个图片给下载下来呢？这里面我们就换一个思路，我们就来抓包。

我们点击Network–XHR，然后我们在往下滑动滚动条时，会一直出现一个名为：acjson?tn=resultjson&ipn=…的请求，点击它再点Preview，我们看到这是一条json数据，点开data，我们看到这里面有30条数据，每一条都对应着一张图片。
![](RE06.png)
于是我们就清楚了，百度图片一开始只加载30张图片，当我们往下滑动滚动条时，页面会动态加载1条json数据，每条json数据里面包含了30条信息，信息里面又包含了图片的URL，JavaScript会将这些url解析并显示出来。这样，每次滚动到底就又多出30张图片。

那么，这些一直出现的json数据有什么规律呢？我们点击Headers，然后对比这些json数据的头部信息。通过对比，我们发现headers下的Query String Parameters中的字段大多保持不变，只有pn字段保持以30为步长递增。wonderful！这下找到规律啦！
![](RE07.png)

## 正文:

既然知道了原理，我们就来写代码实现Python爬虫抓取百度图片。

二话不说就上代码：

In [1]:
import requests
import os

def getManyPages(keyword,pages):
    params=[]
    for i in range(30,30*pages+30,30):
        params.append({
                      'tn': 'resultjson_com',
                      'ipn': 'rj',
                      'ct': 201326592,
                      'is': '',
                      'fp': 'result',
                      'queryWord': keyword,
                      'cl': 2,
                      'lm': -1,
                      'ie': 'utf-8',
                      'oe': 'utf-8',
                      'adpicid': '',
                      'st': -1,
                      'z': '',
                      'ic': 0,
                      'word': keyword,
                      's': '',
                      'se': '',
                      'tab': '',
                      'width': '',
                      'height': '',
                      'face': 0,
                      'istype': 2,
                      'qc': '',
                      'nc': 1,
                      'fr': '',
                      'pn': i,
                      'rn': 30,
                      'gsm': 'f0',
                      '1502723412061': ''
                  })
    url = 'https://image.baidu.com/search/acjson'
    urls = []
    for i in params:
        urls.append(requests.get(url,params=i).json().get('data'))

    return urls


def getImg(dataList, localPath):

    if not os.path.exists(localPath):  # 新建文件夹
        os.mkdir(localPath)

    x = 0
    for list in dataList:
        for i in list:
            if i.get('thumbURL') != None:
                print('正在下载：%s' % i.get('thumbURL'))
                ir = requests.get(i.get('thumbURL'))
                open(localPath + '%d.jpg' % x, 'wb').write(ir.content)
                x += 1
            else:
                print('图片链接不存在')

if __name__ == '__main__':
    dataList = getManyPages('足球',10)  # 参数1:关键字，参数2:要下载的页数
    getImg(dataList,'e:/img17/') # 参数2:指定保存的路径

正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=3524387630,3362326705&fm=26&gp=0.jpg
正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1607424157,11012230&fm=26&gp=0.jpg
正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2069121308,113400979&fm=26&gp=0.jpg
正在下载：https://ss1.bdstatic.com/70cFvXSh_Q1YnxGkpoWK1HF6hhy/it/u=3984101183,2367744588&fm=26&gp=0.jpg
正在下载：https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=2161000909,3098842994&fm=26&gp=0.jpg
正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=3969718753,2606429827&fm=26&gp=0.jpg
正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=3124102422,2686194536&fm=26&gp=0.jpg
正在下载：https://ss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=1704988281,2318513711&fm=200&gp=0.jpg
正在下载：https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2012384539,2163424292&fm=26&gp=0.jpg
正在下载：https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=758356308,733067410&fm=26&gp=0.jpg
正在下载